In [2]:
import cv2

In [3]:
img = cv2.imread("data/9章/img/img01.jpg")
height, width = img.shape[:2]
print("画像幅: " + str(width))
print("画像高さ: " + str(height))
cv2.imshow("img", img)
cv2.waitKey(0)

画像幅: 1920
画像高さ: 1440


124

In [ ]:
cap = cv2.VideoCapture("data/9章/mov/mov01.avi")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print(str(width))
print(str(height))
print(str(count))
print(str(fps))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()